In [0]:
%sql
select * from democatalog.bronze.v_orders

### Pre-Process the JSON String to fix the data Quality Issue

In [0]:
%sql
select value,
       regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') as fixed_value
from democatalog.bronze.v_orders

In [0]:
%sql
create or replace temporary view tv_orders_fixed
as
select value,
       regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') as fixed_value
from democatalog.bronze.v_orders


### Transform JSON String to JSON Object

In [0]:
%sql
select schema_of_json(fixed_value) as schema,
  fixed_value
from tv_orders_fixed
limit 1;

In [0]:
%sql
select from_json(fixed_value,
    'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>'
) as json_value,
  fixed_value
from tv_orders_fixed
limit 1;

In [0]:
%sql
select from_json(fixed_value,
    'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>'
) as json_value,
  fixed_value
from tv_orders_fixed

### Write Transformed Data to the silver schema

In [0]:
%sql
drop table if exists democatalog.silver.orders_json;
create table if not exists democatalog.silver.orders_json
as
select from_json(fixed_value,
    'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>'
) as json_value
from tv_orders_fixed

In [0]:
%sql
select * from democatalog.silver.orders_json